In [ ]:
from os.path import join as pjoin
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from tqdm import tqdm

import tensorflow as tf
import tensorflow_addons as tfa
tfkl = tf.keras.layers
from tqdm.keras import TqdmCallback
import json
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
import time
import math
import os

import matplotlib.pyplot as plt

import warnings
warnings.simplefilter('ignore')

from preprocess_models.full_length import Preprocess
from models.LSTM import get_model
from utils import *

from sklearn.metrics import multilabel_confusion_matrix

# Training loop

In [ ]:
from utils import load_relevant_data_subset
from preprocess_models.interpolated_ragged import Preprocess
from datasets.ragged_sym import get_KFold_dataset
from models.gru import get_model

folds = 7
path = "preprocessed_datasets/interpolated_ragged_nanfinalmask/"

ds = get_KFold_dataset(Preprocess(10), (None, 122), path)
hp = {
    "gru1": 96,
    'nhead': 12,
    'ff_dim': 160,
    'input_dropout': 0.3,
    'gru2': 96,
    'output_dropout': 0.2
}

get_model(hp).summary()

In [ ]:
val_accs = list()  
for fold_idx in range(folds):
    start = time.time()
    print(f'\n            *** Fold {fold_idx} ***\n')
    train_ds = ds.filter(lambda v, l, g, pid, sid: g != fold_idx).map(detuple).padded_batch(32)
    valid_ds = ds.filter(lambda v, l, g, pid, sid: g == fold_idx).map(detuple).padded_batch(32)
    model = get_model(hp)

    lr = 1e-3

    model.compile(
        tf.keras.optimizers.Adam(learning_rate=lr),
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
    )
    
    hist = model.fit(
        x=train_ds,
        epochs=100,
        verbose=2,
        callbacks=[
            TqdmCallback(verbose=0),
            # tfa.callbacks.TQDMProgressBar(),
            # TimeLimitCallback(start, 1, 10), 
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(pjoin(path.split('/')[0], f"model_{fold_idx}"), 
                save_best_only=True, 
                save_weights_only=True,
                restore_best_weights=True, 
                monitor="val_sparse_categorical_accuracy", mode="max"),
            tf.keras.callbacks.EarlyStopping(patience=10, monitor="val_sparse_categorical_accuracy", mode="max", restore_best_weights=True)
            ],
        validation_data=valid_ds,
        validation_freq=1,
        workers=2,
        use_multiprocessing=True
    )

    best_acc = max(hist.history['val_sparse_categorical_accuracy'])
    print("Best acc fold", fold_idx, ":\n ->", 100*round(best_acc, 4), "%")
    val_accs.append(
        best_acc
    )
    # break

print("Bagged final valid acc score:")
bagged_score = 100*np.round(np.array(val_accs).mean(), 4)
print(bagged_score, "%")

globalement 50 s pour une epoch -> voir effet batch size réduit sur généralisation ?

Epoch 47/100

2520/2520 - 52s - loss: 0.5897 - sparse_categorical_accuracy: 0.8558 - val_loss: 1.9944 - val_sparse_categorical_accuracy: 0.5938 - lr: 2.0242e-05 - 52s/epoch - 21ms/step

# Sandbox preprocessing

In [ ]:
left_handed = p(load_relevant_data_subset('/Users/gus/Desktop/envs/asl/asl-signs/train_landmark_files/16069/1101405492.parquet'))
right_handed = p(load_relevant_data_subset('/Users/gus/Desktop/envs/asl/asl-signs/train_landmark_files/28656/55639303.parquet'))

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imageio

# Define the plot function
def plotte(x): 
    plt.plot(x[:, 0], x[:, 1], 'ro')
    plt.xlim(0, 1)
    plt.ylim(0, 1)

# Create a list to store the plot images
images = []

for i, im in enumerate(left_handed[:]):
    # Create a new plot
    plt.figure()
    plotte(im)
    # Save the plot as an image
    plt.savefig(f"tmp/frame_{i:02d}.png")
    
    # Close the plot
    plt.close()

    # Add the image to the list
    images.append(Image.open(f"tmp/frame_{i:02d}.png"))

# Save the images as a GIF file
imageio.mimsave("left_handed.gif", images, fps=5)
len(images)

# Bayesian search template

In [ ]:
from keras_nlp.layers.transformer_encoder import TransformerEncoder
from keras_nlp.layers import SinePositionEncoding

class MyModel(tf.keras.Model):
    def __init__(self, hp):
        super(MyModel, self).__init__()
        self.hp = hp
        self.reshape = tfkl.Reshape((normalized_length, -1))

        self.mean_embedder = tf.keras.Sequential([
            tfkl.Dense(hp['meandim']),
            tfkl.BatchNormalization(),
            tfkl.Activation('gelu')
        ])

        self.std_embedder = tf.keras.Sequential([
            tfkl.Dense(hp["stddim"]),
            tfkl.BatchNormalization(),
            tfkl.Activation('gelu'),
            # tfkl.Dropout(0.3),
        ])

        self.norm = tfkl.BatchNormalization()
        self.gelu = tfkl.Activation('gelu')
        self.transformer_encoders = [TransformerEncoder(
            intermediate_dim=2*hp["d_model"], 
            num_heads=hp["heads"], 
            dropout=hp["dropout"]
            ) for _ in range(hp["modules"])]

        # self.cls_token = tf.Variable(tf.random.normal((1, 1, hp['maindim'])), name='cls')
        self.cls_token = self.add_weight(shape=(1, 1, hp['maindim']), initializer='random_normal', name='cls')

        self.dropout = tfkl.Dropout(hp["dropout2"])
        self.classifier = tfkl.Dense(250, activation="softmax")
    
    def call(self, inputs):
        x = self.reshape(inputs)

        b, t, d = x.shape
        d = int(d/2)
        means = x[:, :, :d]
        stds = x[:, :, d:]
        # Early fusion of mean & std modalities, (and of hands & lips modalities)
        x = tf.concat([self.mean_embedder(means), self.std_embedder(stds)], axis=-1)
        
        # Append cls token
        cls_tokens = tf.tile(self.cls_token, [tf.shape(x)[0], 1, 1])
        x = tfkl.Concatenate(axis=1)([x, cls_tokens])

        # Add positional encoding
        x += SinePositionEncoding()(x)

        x = self.norm(x) # ?
        # x = self.gelu(x)

        # Transformers layers
        for encoder in self.transformer_encoders:
            x = encoder(x)
        
        # CLS pooling
        x = x[:, -1, :]

        # Classification
        x = self.dropout(x)
        x = self.classifier(x)
        
        return x

def get_model(hp):
    main_dim = hp.Int(name='maindim', min_value=32, max_value=256, step=32)
    heads = hp.Int(name='heads', min_value=4, max_value=16, step=4)
    dropout_att = hp.Float(name='dropout', min_value=0.1, max_value=0.5, step=0.1)
    dropout_emb = hp.Float(name='dropout2', min_value=0.1, max_value=0.3, step=0.1)
    lstdim = hp.Int(name='lstmdim', min_value=32, max_value=128, step=32)
    n_modules = hp.Int(name='modules', min_value=1, max_value=4, step=1)
    return MyModel(hp)

model = get_model(hp)
model(e[0]).shape

In [ ]:
from kerastuner.tuners import BayesianOptimization
from keras import backend as backend
backend.clear_session()

val_accs = list()  
for fold_idx in range(folds):
    start = time.time()
    print(f'\n            *** Fold {fold_idx} ***\n')
    train_ds = ds.filter(lambda v, l, g: g != fold_idx).map(detuple).batch(32)
    valid_ds = ds.filter(lambda v, l, g: g == fold_idx).map(detuple).batch(32)
    model = get_model(hp)

    lr = 1e-3

    model.compile(
        tf.keras.optimizers.Adam(learning_rate=lr),
        tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=[
            tf.keras.metrics.SparseCategoricalAccuracy(),
        ]
    )
    
    hist = model.fit(
        x=train_ds,
        epochs=50,
        verbose=2,
        callbacks=[
            TimeLimitCallback(start, 1, 10), 
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(pjoin(path.split('/')[0], f"model_{fold_idx}"), 
                save_best_only=True, 
                save_weights_only=True,
                restore_best_weights=True, 
                monitor="val_sparse_categorical_accuracy", mode="max"),
            tf.keras.callbacks.EarlyStopping(patience=5, monitor="val_sparse_categorical_accuracy", mode="max", restore_best_weights=True)
            ],
        validation_data=valid_ds,
        validation_freq=1,
        workers=2,
        use_multiprocessing=True
    )

    best_acc = max(hist.history['val_sparse_categorical_accuracy'])
    print("Best acc fold", fold_idx, ":\n ->", 100*round(best_acc, 4), "%")
    val_accs.append(
        best_acc
    )

print("Bagged final valid acc score:")
bagged_score = 100*np.round(np.array(val_accs).mean(), 4)
print(bagged_score, "%")

    

In [ ]:
tuner = BayesianOptimization(
    get_model,
    objective='val_sparse_categorical_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='',
    project_name=path,
)

tuner.search(train_ds, epochs=50, validation_data=valid_ds, callbacks=cb_list)